# Small World Graphs

Code examples from [Think Complexity, 2nd edition](https://thinkcomplex.com).

Copyright 2016 Allen Downey, [MIT License](http://opensource.org/licenses/MIT)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import seaborn as sns

from utils import decorate, savefig

# I set the random seed so the notebook 
# produces the same results every time.
np.random.seed(17)

In [ ]:
# node colors for drawing networks
colors = sns.color_palette('pastel', 5)
#sns.palplot(colors)
sns.set_palette(colors)

**Exercise 3.2:** My implementation of `reachable_nodes_bfs` is efficient in the sense that it is in $O(n + m)$, but it incurs a lot of overhead adding nodes to the queue and removing them.  NetworkX provides a simple, fast implementation of BFS, available from [the NetworkX repository on GitHub](https://github.com/networkx/networkx/blob/master/networkx/algorithms/components/connected.py).

Here is a version I modified to return a set of nodes:

In [ ]:
def plain_bfs(G, start):
    """A fast BFS node generator"""
    seen = set()
    nextlevel = {start}
    while nextlevel:
        thislevel = nextlevel
        nextlevel = set()
        for v in thislevel:
            if v not in seen:
                seen.add(v)
                nextlevel.update(G[v])
    return seen

Compare this function to `reachable_nodes_bfs` and see which is faster.  Then see if you can modify this function to implement a faster version of `shortest_path_dijkstra`

In [ ]:
#From exercise 3.1, we use this to generate graphs for testing

#Function from the notebook
def adjacent_edges(nodes, halfk):
    """Yields edges between each node and `halfk` neighbors.
    
    halfk: number of edges from each node
    """
    n = len(nodes)
    for i, u in enumerate(nodes):
        for j in range(i+1, i+halfk+1):
            v = nodes[j % n]
            yield u, v
            
            
def opposite_edge(nodes):
    n = len(nodes)
    halfn = n//2
    
    #We take the opposite node (for even number of nodes) by taking half the length of the list
    #and using that to index the nodes. Taking the modulon guarantees that you are periodically 
    #indexing within the length the list
    
    for i, u in enumerate(nodes):
        index = (i + halfn)%n
        v = nodes[index]
        yield u,v
        


def make_regular_graph(n, k):
    #This function uses adjacent_edges and opposite_edge (when k is odd)
    #Getting the remainder and the quotient is used for checking if
    #k is odd or not.
    
    #A regular graph is a graph that has nodes that have the same number of neighbors
    #In this case, n is the number of nodes and k is the number of neighbors.
    
    quo_k = k//2
    mod_k = k%2
    
    G = nx.Graph()
    nodes = range(n)
    G.add_nodes_from(nodes)
    G.add_edges_from(adjacent_edges(nodes, quo_k))
    
    if mod_k == 1: #when k is odd
        if n%2 == 0: #when nodes are even
            G.add_edges_from(opposite_edge(nodes))
        else:
            raise ValueError("Regular graph cannot be generated if both n and k are both odd.")
            
    return G

In [ ]:
#From chapt 3 notebook
from collections import deque

def reachable_nodes_bfs(G, start):
    """Finds reachable nodes by BFS.
    
    G: graph
    start: node to start at
    
    returns: set of reachable nodes
    """
    seen = set()
    queue = deque([start])
    while queue:
        node = queue.popleft()
        if node not in seen:
            seen.add(node)
            queue.extend(G.neighbors(node))
    return seen

In [ ]:
graph = make_regular_graph(100, 6)

In [ ]:
%timeit reachable_nodes_bfs(graph, 0)

In [ ]:
%timeit plain_bfs(graph, 0)

In [ ]:
graph = make_regular_graph(1000, 6)

In [ ]:
%timeit reachable_nodes_bfs(graph, 0)

In [ ]:
%timeit plain_bfs(graph, 0)

plain_bfs is faster than reachable_nodes_bfs. From plain_bfs, we can write a shortest_path_dijkstra

In [ ]:
### This is the example in the notebook
def shortest_path_dijkstra(G, source):
    """Finds shortest paths from `source` to all other nodes.
    
    G: graph
    source: node to start at
    
    returns: make from node to path length
    """
    dist = {source: 0}
    queue = deque([source])
    while queue:
        node = queue.popleft()
        new_dist = dist[node] + 1

        neighbors = set(G[node]).difference(dist)
        for n in neighbors:
            dist[n] = new_dist
        
        queue.extend(neighbors)
    return dist

By modifying plain_bfs, we can calculate the shortest path length from a starting node to other nodes in the network.

In [ ]:
def shortest_path_length_mod(G, start):
    #Modified plain_bfs to get path lengths instead
    #G is the graph and start is the starting node.
    
    dist_counter = 0 #Starting distance from start
    dist = {}
    nextlevel = {start} #Initialize while loop with the starting node
    
    while nextlevel:
        thislevel = nextlevel
        nextlevel = set()
        for v in thislevel:
            if v not in dist: #only execute if v is not yet in the list of distances
                dist[v] = dist_counter #if the v is not in the dictionary
                nextlevel.update(G[v]) #Update set with neighbors of v
        dist_counter += 1 #add 1 to distance
    
    return dist

In [ ]:
graph = make_regular_graph(100, 4)
# nx.draw_circular(graph, 
#                  node_color='C1', 
#                  node_size=1000, with_labels = True)

In [ ]:
%timeit shortest_path_length_mod(graph, 0)

In [ ]:
%timeit nx.single_source_shortest_path_length(graph, 0)

In [ ]:
%timeit shortest_path_dijkstra(graph, 0)

In [ ]:
graph = make_regular_graph(10, 4)

In [ ]:
shortest_path_length_mod(graph, 0)

In [ ]:
nx.single_source_shortest_path_length(graph, 0)

In [ ]:
shortest_path_dijkstra(graph, 0)

We see that this is quite fast, even faster than the networkx one